In [4]:
import pandas as pd
import numpy as np


In [5]:

import pickle

with open('spy_data_clean.pickle', 'rb') as handle:
  spydf = pickle.load(handle)


In [24]:


def filter_by_time_interval(df, interval_seconds, tolerance_seconds=2):
    """
    Filters a DataFrame to include rows at a specified time interval,
    rounding to the closest data point within a tolerance.

    Args:
        df (pd.DataFrame): The input DataFrame with a 'DateTime' column.
        interval_seconds (int): The desired time interval in seconds.
        tolerance_seconds (int): The tolerance in seconds for rounding.

    Returns:
        pd.DataFrame: The filtered DataFrame.
    """

    # Ensure DateTime column is sorted
    df = df.sort_values(by=['DateTime'])

    # Calculate the time difference between consecutive rows
    df['TimeDiff'] = df['DateTime'].diff().dt.total_seconds()

    # Initialize variables for tracking the last selected row and current time
    last_selected_time = pd.NaT
    selected_indices = []

    for index, row in df.iterrows():
        current_time = row['DateTime']

        # Check if it's the first row or if enough time has passed since the last selected row
        if pd.isnull(last_selected_time) or (current_time - last_selected_time).total_seconds() >= (interval_seconds - tolerance_seconds):

            # Round to the nearest interval using numpy.around
            rounded_time_diff = np.around((current_time - last_selected_time).total_seconds() / interval_seconds) * interval_seconds if not pd.isnull(last_selected_time) else np.nan
            # print(rounded_time_diff)

            # Check if the rounded time difference is within the tolerance
            if pd.isnull(rounded_time_diff) or (abs(rounded_time_diff - (current_time - last_selected_time).total_seconds()) <= tolerance_seconds):
                selected_indices.append(index)
                last_selected_time = current_time


    # Select the rows based on the selected indices
    filtered_df = df.loc[selected_indices].copy()

    # Drop the temporary 'TimeDiff' column
    filtered_df = filtered_df.drop(columns=['TimeDiff'])

    return filtered_df

# Example usage:


In [28]:
newdf=spydf.iloc[100000:101000]

In [29]:
filtered_df = filter_by_time_interval(newdf, 5, tolerance_seconds=2)
print(filtered_df)

         price            DateTime
100000  1426.5 2000-02-07 10:43:40
100003  1426.3 2000-02-07 10:43:52
100004  1426.4 2000-02-07 10:43:55
100005  1426.3 2000-02-07 10:44:01
100006  1426.5 2000-02-07 10:44:05
...        ...                 ...
100993  1423.2 2000-02-07 12:26:13
100995  1423.2 2000-02-07 12:26:26
100997  1423.2 2000-02-07 12:26:36
100998  1423.5 2000-02-07 12:26:40
100999  1423.3 2000-02-07 12:26:48

[660 rows x 2 columns]


In [30]:
len(filtered_df)

660

In [31]:
# prompt: plot filtered_df using plotly

import plotly.express as px

# Assuming filtered_df is defined as in your previous code

# Create a Plotly figure
fig = px.line(filtered_df, x='DateTime', y='price')

# Customize the plot (optional)
fig.update_layout(
    title='Filtered SPY Close Prices',
    xaxis_title='Date and Time',
    yaxis_title='Close Price'
)

# Show the plot
fig.show()

In [32]:
# prompt: using the filtered data, calculate the veloctiy of as [P(t)-P(t-1)]/t and acceleration as [V(t)-V(t-1)]/t append those as columns to a new data frame. Make sure no out of bounds error. Since veloctiy is use price at current point -1 and etc ensure that doenst casue an error, limit it to the second poitn

# Assuming filtered_df is defined as in your previous code

# Create new columns for velocity and acceleration
filtered_df['velocity'] = np.nan
filtered_df['acceleration'] = np.nan

# Calculate velocity and acceleration
for i in range(1, len(filtered_df)):
  # Calculate velocity
  current_price = filtered_df['price'].iloc[i]
  previous_price = filtered_df['price'].iloc[i-1]
  time_diff = (filtered_df['DateTime'].iloc[i] - filtered_df['DateTime'].iloc[i-1]).total_seconds()

  if time_diff > 0:  # Prevent division by zero
    filtered_df.loc[filtered_df.index[i], 'velocity'] = (current_price - previous_price) / time_diff

  # Calculate acceleration only if velocity is available for current and previous point.
  if i > 1 and not np.isnan(filtered_df['velocity'].iloc[i]) and not np.isnan(filtered_df['velocity'].iloc[i-1]):
    current_velocity = filtered_df['velocity'].iloc[i]
    previous_velocity = filtered_df['velocity'].iloc[i-1]

    filtered_df.loc[filtered_df.index[i], 'acceleration'] = (current_velocity - previous_velocity) / time_diff


filtered_df

,price,DateTime,velocity,acceleration
100000,1426.5,2000-02-07 10:43:40,NaN,NaN
100003,1426.3,2000-02-07 10:43:52,-0.016667,NaN
100004,1426.4,2000-02-07 10:43:55,0.033333,0.016667
100005,1426.3,2000-02-07 10:44:01,-0.016667,-0.008333
100006,1426.5,2000-02-07 10:44:05,0.050000,0.016667
...,...,...,...,...
100993,1423.2,2000-02-07 12:26:13,0.028571,0.001327
100995,1423.2,2000-02-07 12:26:26,0.000000,-0.002198
100997,1423.2,2000-02-07 12:26:36,0.000000,0.000000
100998,1423.5,2000-02-07 12:26:40,0.075000,0.018750


In [33]:
# prompt: add another colomun to filtered df, Direction, if the next point's price is larger then the current set it to 1, otherwise 0

# Assuming filtered_df is defined as in your previous code

# Create a new column 'Direction' and initialize it with 0
filtered_df['Direction'] = 0

# Iterate through the rows and calculate the direction
for i in range(len(filtered_df) - 1):
  if filtered_df['price'].iloc[i+1] > filtered_df['price'].iloc[i]:
    filtered_df.at[filtered_df.index[i], 'Direction'] = 1

filtered_df

,price,DateTime,velocity,acceleration,Direction
100000,1426.5,2000-02-07 10:43:40,NaN,NaN,0
100003,1426.3,2000-02-07 10:43:52,-0.016667,NaN,1
100004,1426.4,2000-02-07 10:43:55,0.033333,0.016667,0
100005,1426.3,2000-02-07 10:44:01,-0.016667,-0.008333,1
100006,1426.5,2000-02-07 10:44:05,0.050000,0.016667,1
...,...,...,...,...,...
100993,1423.2,2000-02-07 12:26:13,0.028571,0.001327,0
100995,1423.2,2000-02-07 12:26:26,0.000000,-0.002198,0
100997,1423.2,2000-02-07 12:26:36,0.000000,0.000000,1
100998,1423.5,2000-02-07 12:26:40,0.075000,0.018750,0


In [34]:
fig = px.line(filtered_df, x='DateTime', y='price',
                 title='Price vs Time with Velocity and Acceleration on Hover',
                 labels={'time': 'Time', 'price': 'Price'},
                 hover_data={'velocity': True, 'acceleration': True,'Direction': True})

# Show plot
fig.show()

In [40]:
fig = px.line(newdf, x='DateTime', y='price')

# Customize the plot (optional)
fig.update_layout(
    title='Filtered SPY Close Prices',
    xaxis_title='Date and Time',
    yaxis_title='Close Price'
)

# Show the plot
fig.show()

In [35]:
# prompt: for points with a direction of 1 find the mean velocity and acceleration

# Assuming filtered_df is defined as in your previous code

# Filter the DataFrame for rows where 'Direction' is 1
direction_1_df = filtered_df[filtered_df['Direction'] == 1]

# Calculate the mean velocity and acceleration for the filtered DataFrame
mean_velocity = direction_1_df['velocity'].mean()
mean_acceleration = direction_1_df['acceleration'].mean()

print(f"Mean velocity for points with Direction 1: {mean_velocity}")
print(f"Mean acceleration for points with Direction 1: {mean_acceleration}")

Mean velocity for points with Direction 1: -0.0005892534691108644
Mean acceleration for points with Direction 1: 0.0009940287425396186


In [36]:
selected_rows = filtered_df[(filtered_df['velocity'] > 0) & (filtered_df['acceleration'] > 0)]


In [37]:
selected_rows.head()

,price,DateTime,velocity,acceleration,Direction
100004,1426.4,2000-02-07 10:43:55,0.033333,0.016667,0
100006,1426.5,2000-02-07 10:44:05,0.050000,0.016667,1
100009,1427.0,2000-02-07 10:44:36,0.018750,0.002214,0
100012,1427.0,2000-02-07 10:45:00,0.011765,0.002373,1
100013,1427.1,2000-02-07 10:45:08,0.012500,0.000092,0


In [38]:
len(selected_rows[selected_rows['Direction'] == 1])/len(selected_rows)

0.45652173913043476